In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy as sp
from scipy import sparse
from anndata import AnnData
import gc
from pynndescent import NNDescent

In [ ]:
def readAndCleanData( peakDataPath, peakMetaPath=None, SEACellPath=None ):
    adataPeak = sc.read_h5ad( peakDataPath )
    if peakMetaPath is not None:
        peaks = pd.read_csv( peakMetaPath, index_col=0 )
        peakdf.index = peakdf['seqnames'] + ':' + \
                       peakdf['start'].astype(str) + '-' + \
                       peakdf['end'].astype(str)
        adataPeak.var = peaks.copy()
    if SEACellPath is not None:
        annots = pd.read_csv( SEACellPath, index_col=0 )
        if 'SEACell' not in adataPeak.obs.columns:
            adataPeak.obs['SEACell'] = annots.loc[adataPeak.obs_names,'SEACell']
        if 'MaxSEACellScore' not in adataPeak.obs.columns:
            adataPeak.obs['MaxSEACellScore'] = annots.loc[adataPeak.obs_names,'MaxSEACellScore']
    adataPeak = adataPeak[adataPeak.obs.MaxSEACellScore>0.1,:].copy()
    return adataPeak

In [ ]:
def readAndAlignMotifHits( rawPath, filteredPath ):
    adataPM = sc.read_h5ad( rawPath )
    adataPMF = sc.read_h5ad( filteredPath )
    # reorder the columns in case they get mixed up
    adataPMF = adataPMF[:,adataPM.var_names].copy()
    intersPeaks = adataPM.obs_names[adataPM.obs_names.isin(adataPMF.obs_names)]
    tmp = np.zeros_like( adataPM.X )
    tmp[adataPM.obs_names.isin(adataPMF.obs_names),:] = adataPMF[intersPeaks,:].X.A
    adataPM.layers['filtered'] = sparse.csr_matrix( tmp )
    return adataPM

In [ ]:
def aggregateCountsOverSEACells( adata ):
    scNames = np.sort(adata.obs.SEACell.unique())
    samples = np.array([n.split('-')[1] for n in scNames])
    X = np.zeros( (scNames.size,adata.n_vars) )

    print( 'Aggregating peak counts over SEACells' )
    for i, name in tqdm(enumerate(scNames)):
        X[i,:] = adata[adata.obs.SEACell==name,:].X.A.sum(0)

    return AnnData( X=sparse.csr_matrix(X),
                    obs=pd.DataFrame( index=scNames, 
                                      data={ 'Sample':samples } ),
                    var=pd.DataFrame( index=adataPeak.var_names ) )

In [ ]:
def calculateChromVARDeviations( adataCounts, adataAnnot, bkgd, annotLayer=None ):
    # Get expected number of reads per peak
    # if per-nucleus depth were only variable
    E = np.zeros( adataCounts.shape )
    samples = adataCounts.obs.Sample.values
    # Do this separately per library
    for s in np.unique(samples):
        print( 'Calculating expectations for sample {0}'.format( s ) )
        mask = samples == s
        # Pull out counts for this sample
        x = adataCounts.X[mask,:].A
        # Multiply depth of each cell by the 
        # frequency of each peak in the whole matrix
        E[mask,:] = x.sum(0)[None,:] / x.sum() * x.sum(1)[:,None]

    # Read in binarized motif annotations
    M = (adataAnnot.X>0).astype(int).T
    if annotLayer is not None:
        M = (adataAnnot.layers[annotLayer].X>0).astype(int).T
    # Count up number of reads in peaks 
    # containing each motif, per cell
    obs = M.dot( adataCounts.X.A.T )
    # Do the same thing based on the null model
    exp = M.dot( E.T )
    den = exp.copy()
    den[den==0] = 1
    # Calculate the percent difference
    # these are the deviations
    Yobs = ( obs - exp ) / den
    
    # Now repeat with GC-matched background peaks
    Ybgd = np.zeros( (bkgd.shape[1],*Yobs.shape) )

    for i in tqdm(range(bkgd.shape[1])):
        # Get motif annotations of GC matched peaks
        shufInd = bkgd[:,i].flatten()
        Mshuf = M[:,shufInd]
        # Calculate the motif hits with shuffled
        # annotations as above
        obs = Mshuf.dot( X.T )
        exp = Mshuf.dot( E.T )
        den = exp.copy()
        den[den==0] = 1
        Ybgd[i,:,:] = ( obs - exp ) / den
        
    # Normalize by dividing each cell x motif
    # observation by the standard deviation
    # of these shuffled trials and 
    # subtracting out the mean
    den = Ybgd.std(0)
    den[den==0] = 1
    Ynorm = ( Yobs - Ybgd.mean(0) ) / den
    Ynorm = np.nan_to_num( Ynorm, 0 )
    # This is now the 'deviation z-scores'
    # from the original ChromVAR paper
    
    return Ynorm

# *M. lignano*

In [ ]:
# Read in the peak accessibilities, annotated by SEACells
adataPeak = readAndCleanData( 'ArchROutputs/Mlig/Mlig.peaks.h5ad',
                              'ArchROutputs/Mlig/Mlig.peaks.csv',
                              'SEACellsOutput/Mlig.all_SEACell_assignments.csv' )

In [ ]:
# Read in the filtered peak-motif annotations
adataPM = readAndAlignMotifHits( 'FilteredPeakMotifHits/Mlig.raw_peak_motif_hits.h5ad',
                                 'FilteredPeakMotifHits/Mlig.filt_peak_motif_hits.h5ad' )

In [ ]:
adataPeak = aggregateCountsOverSEACells( adataPeak )

In [ ]:
# (N,50) array of 50 GC matched peak indices per peak
bgdPeaks = np.loadtxt( 'ArchROutputs/Mlig/Mlig.bgd_peaks.txt' ).astype(int) - 1

In [ ]:
Xdev = calculateChromVARDeviations( adataPeak, adataPM, annotLayer='filtered' )

In [ ]:
AnnData( X=Xdev.T, obs=adataPeak.obs, var=adataPM.var )\
    .write_h5ad( 'ChromVARDeviations/Mlig.SEACells_devs.h5ad' )

# *S. mediterranea*

In [ ]:
# Read in the peak accessibilities, annotated by SEACells
adataPeak = readAndCleanData( 'ArchROutputs/Smed/Smed.peaks.h5ad',
                              'ArchROutputs/Smed/Smed.peaks.csv',
                              'SEACellsOutput/Smed.all_SEACell_assignments.csv' )

In [ ]:
# Read in the filtered peak-motif annotations
adataPM = readAndAlignMotifHits( 'FilteredPeakMotifHits/Smed.raw_peak_motif_hits.h5ad',
                                 'FilteredPeakMotifHits/Smed.filt_peak_motif_hits.h5ad' )

In [ ]:
adataPeak = aggregateCountsOverSEACells( adataPeak )

In [ ]:
# (N,50) array of 50 GC matched peak indices per peak
bgdPeaks = np.loadtxt( 'ArchROutputs/Smed/Smed.bgd_peaks.txt' ).astype(int) - 1

In [ ]:
Xdev = calculateChromVARDeviations( adataPeak, adataPM, annotLayer='filtered' )

In [ ]:
AnnData( X=Xdev.T, obs=adataPeak.obs, var=adataPM.var )\
    .write_h5ad( 'ChromVARDeviations/Mlig.SEACells_devs.h5ad' )

# *S. mansoni*

In [ ]:
# Read in the peak accessibilities, annotated by SEACells
adataPeak = readAndCleanData( 'ArchROutputs/Sman/Sman.peaks.h5ad',
                              'ArchROutputs/Sman/Sman.peaks.csv',
                              'SEACellsOutput/Sman.all_SEACell_assignments.csv' )

In [ ]:
# Read in the filtered peak-motif annotations
adataPM = readAndAlignMotifHits( 'FilteredPeakMotifHits/Sman.raw_peak_motif_hits.h5ad',
                                 'FilteredPeakMotifHits/Sman.filt_peak_motif_hits.h5ad' )

In [ ]:
adataPeak = aggregateCountsOverSEACells( adataPeak )

In [ ]:
# (N,50) array of 50 GC matched peak indices per peak
bgdPeaks = np.loadtxt( 'ArchROutputs/Sman/Sman.bgd_peaks.txt' ).astype(int) - 1

In [ ]:
Xdev = calculateChromVARDeviations( adataPeak, adataPM, annotLayer='filtered' )

In [ ]:
AnnData( X=Xdev.T, obs=adataPeak.obs, var=adataPM.var )\
    .write_h5ad( 'ChromVARDeviations/Mlig.SEACells_devs.h5ad' )